In [1]:
import pandas as pd
from airflow.hooks.mssql_hook import MsSqlHook
from poseidon.util import general
from poseidon.dags.streets.streets_jobs import *
import requests
from datetime import datetime, timedelta



conf = general.config

pd.options.display.max_columns = 999



[2017-04-21 20:44:12,716] {__init__.py:36} INFO - Using executor SequentialExecutor
[2017-04-21 20:44:12,849] {driver.py:120} INFO - Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt
[2017-04-21 20:44:12,913] {driver.py:120} INFO - Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


In [4]:
"""Get streets paving data from DB."""
mode = "imcat"
pv_query = general.file_to_string('../sql/pavement_ex.sql', '__file__')
pv_conn = MsSqlHook(mssql_conn_id='streets_cg_sql')

moratorium_string = "Post Construction"
phone_UTLY = "858-627-3200"
phone_OTHER = "619-527-7500"
TSW_PM = "JLahmann@sandiego.gov"
UTLY_PM = "Engineering@sandiego.gov"
ACT_OVERLAY_CONCRETE_PM = "CHudson@sandiego.gov"
ACT_SLURRY_SERIES_PM = "JJaro@sandiego.gov"

# Different String for imcat mode.
if mode == 'imcat':
    moratorium_string = "Post-Construction"

df = pv_conn.get_pandas_df(pv_query)

df.head()


[2017-03-22 17:44:14,184] {base_hook.py:53} INFO - Using connection to: csdldcpvvsql01.sannet.gov


,pve_id,seg_id,rd_seg_id,wo_id,wo_name,wo_pm,wo_pm_phone,wo_design_start_dt,wo_design_end_dt,job_start_dt,job_end_dt,job_completed_cbox,wo_status,wo_proj_type,job_activity,wo_resident_engineer,street,street_from,street_to,job_entry_dt,job_updated_dt,seg_placed_in_srvc,seg_func_class,seg_placed_in_service,seg_council_district,seg_length_ft,seg_width_ft
0,140392,SS-017569,5902.0,AC1501,AC OVERLAY GROUP 1501,CHudson@sandiego.gov,619-527-8081,2015-02-02,2015-07-15,2015-02-11,2015-02-11,1.0,POST-CONSTRUCTION,Overlay,AC - AC Overlay,Jeff Wooley,MARKET ST,06TH AV,07TH AV,NaT,2015-02-20 09:47:26,2002-11-18,Major,2002-11-18,3,277.0073,80.0
1,41189350,SS-007546,17466.0,AC1501,AC OVERLAY GROUP 1501,CHudson@sandiego.gov,619-527-8081,2015-02-02,2015-07-15,2015-04-30,2015-04-30,1.0,POST-CONSTRUCTION,Overlay,AC - AC Overlay,Jeff Wooley,COLLEGE AV,CAM RICO,NAVAJO RD,2012-04-16 07:56:17,2015-05-05 13:03:18,1968-06-01,Major,1968-06-01,7,532.5487,70.0
2,60681043,SS-028267,63851.0,AC1501,AC OVERLAY GROUP 1501,CHudson@sandiego.gov,619-527-8081,2015-02-02,2015-07-15,2015-04-27,2015-04-27,1.0,POST-CONSTRUCTION,Overlay,AC - AC Overlay,Jeff Wooley,WARING RD,GREENBRIER AV,MOUND AV,2012-09-12 14:07:01,2015-04-30 13:15:16,1963-06-01,Major,1963-06-01,7,1132.5570,60.0
3,588445703,SS-020550,67352.0,AC1501,AC OVERLAY GROUP 1501,CHudson@sandiego.gov,619-527-8081,2015-02-02,2015-07-15,2015-05-20,2015-05-20,1.0,POST-CONSTRUCTION,Overlay,AC - AC Overlay,Jeff Wooley,OMEGA DR,PARTHENON DR,SYLVY WY,2011-07-19 13:14:46,2015-05-27 13:40:13,1975-06-01,Residential,1975-06-01,4,209.1024,40.0
4,614134820,SS-022123,53291.0,AC1501,AC OVERLAY GROUP 1501,CHudson@sandiego.gov,619-527-8081,2015-02-02,2015-07-15,2015-03-25,2015-03-25,1.0,POST-CONSTRUCTION,Overlay,AC - AC Overlay,Jeff Wooley,POLK AV,LOUISIANA ST,TEXAS ST,2011-12-15 09:19:47,2015-03-30 10:48:20,1954-06-01,Residential,1954-06-01,3,380.0052,52.0


In [14]:

for i in [
        'seg_id', 'rd_seg_id', 'wo_id', 'wo_name', 'wo_status',
        'wo_proj_type', 'job_activity', 'seg_func_class'
]:

    df[i] = df[i].astype(str)

df['job_completed_cbox'] = df['job_completed_cbox'].astype(bool)

# Backfill - set all fields to mora
df.loc[df.wo_status.str.contains(
    'post construction|moratorium|post-construction',
    regex=True,
    case=False), "wo_status"] = moratorium_string

# Remove Records w/o A Completed Date ONLY in the UTLY and TSW work order
# IMCAT ONLY
if mode == 'imcat':
    df = df.query('not '\
                + '((wo_id == "UTLY" & job_end_dt.isnull()) '\
                + 'or (wo_id == "TSW" & job_end_dt.isnull()))')

    # Remove empty activities (IMCAT ONLY)
    df = df.query('not '\
                + '(job_activity.isnull() '\
                + '| job_activity == "" '\
                + '| job_activity == "None")')

# Remove Data Entry
# Remove mill / pave
# Remove Structure Widening
# Remove Patching
if mode == 'imcat':
    remove_search = 'data entry|mill|structure wid|patching'
else:
    remove_search = 'data entry|structure wid|patching'

df = df[~(df.job_activity.str.contains(
    remove_search, regex=True, case=False))]

# Search Strings
concrete_search = "panel rep|pcc - reconstruc"
slurry_search = "surface treatment|scrub seal|cape seal|central mix"
overlay_search = "resurfacing|overlay|ac overlay|mill|ac - overlay|ac - ac overlay|ac - reconstruct|ac inlay"

# Set Proj Type to NA
df['wo_proj_type'] = None
# Concrete
df.loc[df.job_activity.str.contains(
    concrete_search, regex=True, case=False), 'wo_proj_type'] = 'Concrete'
# Slurry
df.loc[df.job_activity.str.contains(
    slurry_search, regex=True, case=False), 'wo_proj_type'] = 'Slurry'
# Overlay
df.loc[df.job_activity.str.contains(
    overlay_search, regex=True, case=False), 'wo_proj_type'] = 'Overlay'

# Remove All Records over 5 Years Old;
#pv <- pv[(as.Date(pv$job_end_dt) > (today() - years(5))) | is.na(pv$job_end_dt),]
from datetime import datetime, timedelta
# Create ref dates
today = datetime.now()
five_yrs_ago = today.replace(year=(today.year - 5))
three_yrs_ago = today.replace(year=(today.year - 3))

# Remove records
df = df[(df.job_end_dt > five_yrs_ago) | (df.job_end_dt.isnull())]

# Remove Slurry Records > 3 Years Old
# IMCAT ONLY
if mode == 'imcat':
    mask = ~((df.wo_proj_type == 'Slurry') &
             (df.job_end_dt < three_yrs_ago))
    df = df[mask]

# Create a feature for completed jobs
df['final_job_completion_state'] = False
#pv[(!is.na(pv$job_end_dt) & pv$job_completed_cbox == 1), "final_job_completion_state"] <- 1

df.loc[df.job_end_dt.notnull(), "final_job_completion_state"] = True

# Set all completed jobs to Moratorium status
df.loc[df.final_job_completion_state == True,
       "wo_status"] = moratorium_string

# Set Dates in The future for TSW work orders as Construction.
mask = (df.wo_id == 'TSW') & \
       (df.job_end_dt.notnull()) & \
       (df.job_end_dt > today)

df.loc[mask, "wo_status"] = "Construction"

In [15]:
df[df['wo_id'] == 'AC1512-JOC8']

,pve_id,seg_id,rd_seg_id,wo_id,wo_name,wo_pm,wo_pm_phone,wo_design_start_dt,wo_design_end_dt,job_start_dt,job_end_dt,job_completed_cbox,wo_status,wo_proj_type,job_activity,wo_resident_engineer,street,street_from,street_to,job_entry_dt,job_updated_dt,seg_placed_in_srvc,seg_func_class,seg_placed_in_service,seg_council_district,seg_length_ft,seg_width_ft,final_job_completion_state
1405,2061867672,SS-012805,70356.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,GUINCHO RD,ANTIGUA BL,HERMANOS RD,2008-04-29 10:46:25,2017-03-15 13:36:13,1973-06-01,Residential,1973-06-01,7,260.9711,36.0,False
1406,2063731208,SS-018035,69573.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-06-25,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MENKAR RD,REVELSTOKE WY,CAPCANO RD,2013-06-25 11:01:36,2016-05-20 15:38:41,1974-01-01,Major,1974-01-01,6,208.2402,40.0,False
1407,474554725,SS-006990,17999.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2012-09-12,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,CHATEAU DR,BOXWOOD DR,CHATEAU PL,2012-09-12 12:47:37,2017-01-11 10:12:33,1961-06-01,Residential,1961-06-01,6,510.6220,40.0,False
1408,624760835,SS-017456,78746.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MARBURY AV,CHART HOUSE ST,MIRA MESA BL,2012-05-29 06:46:03,2016-04-26 10:07:44,1971-06-01,Residential,1971-06-01,6,540.4199,40.0,False
1409,625445553,SS-010139,65576.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-02-19,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,ELDERWOOD LN,SPRING CANYON RD,ELDERWOOD CT,2013-02-19 17:11:23,2016-04-25 16:29:47,1986-06-01,Residential,1986-06-01,5,618.3855,36.0,False
1410,703454813,SS-010086,70358.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,EL HONCHO PL,AMARO DR,END,2009-11-03 13:12:15,2016-04-25 16:27:40,1973-06-01,Residential,1973-06-01,7,492.7223,36.0,False
1411,694296983,SS-015055,69656.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2016-02-29,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,KELOWNA RD,SCHNEPLE DR,DUNCANNON CT,2016-02-29 14:09:50,2016-04-26 09:24:14,1975-06-01,Residential,1975-06-01,6,264.0001,40.0,False
1412,767285743,SS-010138,65578.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-02-19,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,ELDERWOOD LN,ELDERWOOD CT,ELDERWOOD RD,2013-02-19 17:11:12,2016-05-04 08:13:03,1986-06-01,Residential,1986-06-01,5,335.9501,36.0,False
1413,366530475,SS-018039,69588.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-06-25,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MENKAR RD,ALKAID DR,BELLATRIX CT,2013-06-25 11:02:27,2016-10-10 15:59:45,1973-06-01,Major,1973-06-01,6,206.0070,40.0,False
1414,125752382,SS-003607,76991.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2014-07-23,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,BAROQUE LN,DUCOS PL,PAGUERA CT,2014-07-23 10:22:14,2016-10-17 07:54:04,1978-06-01,Residential,1978-06-01,7,165.5518,36.0,False


In [16]:
# Set Phone # For UTLY
df.loc[df.wo_id == 'UTLY', 'wo_pm_phone'] = phone_UTLY

# Set Phone # for Everything else
df.loc[df.wo_id != 'UTLY', 'wo_pm_phone'] = phone_OTHER

# Set PM for UTLY
df.loc[df.wo_id == 'UTLY', 'wo_pm'] = UTLY_PM

# Set PM for TSW
df.loc[df.wo_id == 'TSW', 'wo_pm'] = TSW_PM
df[df['wo_id'] == 'AC1512-JOC8']

,pve_id,seg_id,rd_seg_id,wo_id,wo_name,wo_pm,wo_pm_phone,wo_design_start_dt,wo_design_end_dt,job_start_dt,job_end_dt,job_completed_cbox,wo_status,wo_proj_type,job_activity,wo_resident_engineer,street,street_from,street_to,job_entry_dt,job_updated_dt,seg_placed_in_srvc,seg_func_class,seg_placed_in_service,seg_council_district,seg_length_ft,seg_width_ft,final_job_completion_state
1405,2061867672,SS-012805,70356.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,GUINCHO RD,ANTIGUA BL,HERMANOS RD,2008-04-29 10:46:25,2017-03-15 13:36:13,1973-06-01,Residential,1973-06-01,7,260.9711,36.0,False
1406,2063731208,SS-018035,69573.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-06-25,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MENKAR RD,REVELSTOKE WY,CAPCANO RD,2013-06-25 11:01:36,2016-05-20 15:38:41,1974-01-01,Major,1974-01-01,6,208.2402,40.0,False
1407,474554725,SS-006990,17999.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2012-09-12,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,CHATEAU DR,BOXWOOD DR,CHATEAU PL,2012-09-12 12:47:37,2017-01-11 10:12:33,1961-06-01,Residential,1961-06-01,6,510.6220,40.0,False
1408,624760835,SS-017456,78746.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MARBURY AV,CHART HOUSE ST,MIRA MESA BL,2012-05-29 06:46:03,2016-04-26 10:07:44,1971-06-01,Residential,1971-06-01,6,540.4199,40.0,False
1409,625445553,SS-010139,65576.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-02-19,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,ELDERWOOD LN,SPRING CANYON RD,ELDERWOOD CT,2013-02-19 17:11:23,2016-04-25 16:29:47,1986-06-01,Residential,1986-06-01,5,618.3855,36.0,False
1410,703454813,SS-010086,70358.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,EL HONCHO PL,AMARO DR,END,2009-11-03 13:12:15,2016-04-25 16:27:40,1973-06-01,Residential,1973-06-01,7,492.7223,36.0,False
1411,694296983,SS-015055,69656.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2016-02-29,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,KELOWNA RD,SCHNEPLE DR,DUNCANNON CT,2016-02-29 14:09:50,2016-04-26 09:24:14,1975-06-01,Residential,1975-06-01,6,264.0001,40.0,False
1412,767285743,SS-010138,65578.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-02-19,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,ELDERWOOD LN,ELDERWOOD CT,ELDERWOOD RD,2013-02-19 17:11:12,2016-05-04 08:13:03,1986-06-01,Residential,1986-06-01,5,335.9501,36.0,False
1413,366530475,SS-018039,69588.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-06-25,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MENKAR RD,ALKAID DR,BELLATRIX CT,2013-06-25 11:02:27,2016-10-10 15:59:45,1973-06-01,Major,1973-06-01,6,206.0070,40.0,False
1414,125752382,SS-003607,76991.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2014-07-23,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,BAROQUE LN,DUCOS PL,PAGUERA CT,2014-07-23 10:22:14,2016-10-17 07:54:04,1978-06-01,Residential,1978-06-01,7,165.5518,36.0,False


In [17]:
df[df['wo_id'] == 'AC1512-JOC8']

,pve_id,seg_id,rd_seg_id,wo_id,wo_name,wo_pm,wo_pm_phone,wo_design_start_dt,wo_design_end_dt,job_start_dt,job_end_dt,job_completed_cbox,wo_status,wo_proj_type,job_activity,wo_resident_engineer,street,street_from,street_to,job_entry_dt,job_updated_dt,seg_placed_in_srvc,seg_func_class,seg_placed_in_service,seg_council_district,seg_length_ft,seg_width_ft,final_job_completion_state
1405,2061867672,SS-012805,70356.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,GUINCHO RD,ANTIGUA BL,HERMANOS RD,2008-04-29 10:46:25,2017-03-15 13:36:13,1973-06-01,Residential,1973-06-01,7,260.9711,36.0,False
1406,2063731208,SS-018035,69573.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-06-25,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MENKAR RD,REVELSTOKE WY,CAPCANO RD,2013-06-25 11:01:36,2016-05-20 15:38:41,1974-01-01,Major,1974-01-01,6,208.2402,40.0,False
1407,474554725,SS-006990,17999.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2012-09-12,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,CHATEAU DR,BOXWOOD DR,CHATEAU PL,2012-09-12 12:47:37,2017-01-11 10:12:33,1961-06-01,Residential,1961-06-01,6,510.6220,40.0,False
1408,624760835,SS-017456,78746.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MARBURY AV,CHART HOUSE ST,MIRA MESA BL,2012-05-29 06:46:03,2016-04-26 10:07:44,1971-06-01,Residential,1971-06-01,6,540.4199,40.0,False
1409,625445553,SS-010139,65576.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-02-19,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,ELDERWOOD LN,SPRING CANYON RD,ELDERWOOD CT,2013-02-19 17:11:23,2016-04-25 16:29:47,1986-06-01,Residential,1986-06-01,5,618.3855,36.0,False
1410,703454813,SS-010086,70358.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,EL HONCHO PL,AMARO DR,END,2009-11-03 13:12:15,2016-04-25 16:27:40,1973-06-01,Residential,1973-06-01,7,492.7223,36.0,False
1411,694296983,SS-015055,69656.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2016-02-29,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,KELOWNA RD,SCHNEPLE DR,DUNCANNON CT,2016-02-29 14:09:50,2016-04-26 09:24:14,1975-06-01,Residential,1975-06-01,6,264.0001,40.0,False
1412,767285743,SS-010138,65578.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-02-19,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,ELDERWOOD LN,ELDERWOOD CT,ELDERWOOD RD,2013-02-19 17:11:12,2016-05-04 08:13:03,1986-06-01,Residential,1986-06-01,5,335.9501,36.0,False
1413,366530475,SS-018039,69588.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-06-25,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MENKAR RD,ALKAID DR,BELLATRIX CT,2013-06-25 11:02:27,2016-10-10 15:59:45,1973-06-01,Major,1973-06-01,6,206.0070,40.0,False
1414,125752382,SS-003607,76991.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2014-07-23,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,BAROQUE LN,DUCOS PL,PAGUERA CT,2014-07-23 10:22:14,2016-10-17 07:54:04,1978-06-01,Residential,1978-06-01,7,165.5518,36.0,False


In [18]:
# Set PM for Overlay / Concrete
mask = (df.wo_pm.isnull()) & ((df.wo_proj_type == 'Overlay') |
                              (df.wo_proj_type == 'Concrete'))
df.loc[mask, 'wo_pm'] = ACT_OVERLAY_CONCRETE_PM

# Set PM for Slurry / Series
mask = (df.wo_pm.isnull()) & ((df.wo_proj_type == 'Slurry') |
                              (df.wo_proj_type == 'Series Circuit'))
df.loc[mask, 'wo_pm'] = ACT_SLURRY_SERIES_PM

df[df['wo_id'] == 'AC1512-JOC8']

,pve_id,seg_id,rd_seg_id,wo_id,wo_name,wo_pm,wo_pm_phone,wo_design_start_dt,wo_design_end_dt,job_start_dt,job_end_dt,job_completed_cbox,wo_status,wo_proj_type,job_activity,wo_resident_engineer,street,street_from,street_to,job_entry_dt,job_updated_dt,seg_placed_in_srvc,seg_func_class,seg_placed_in_service,seg_council_district,seg_length_ft,seg_width_ft,final_job_completion_state
1405,2061867672,SS-012805,70356.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,GUINCHO RD,ANTIGUA BL,HERMANOS RD,2008-04-29 10:46:25,2017-03-15 13:36:13,1973-06-01,Residential,1973-06-01,7,260.9711,36.0,False
1406,2063731208,SS-018035,69573.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-06-25,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MENKAR RD,REVELSTOKE WY,CAPCANO RD,2013-06-25 11:01:36,2016-05-20 15:38:41,1974-01-01,Major,1974-01-01,6,208.2402,40.0,False
1407,474554725,SS-006990,17999.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2012-09-12,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,CHATEAU DR,BOXWOOD DR,CHATEAU PL,2012-09-12 12:47:37,2017-01-11 10:12:33,1961-06-01,Residential,1961-06-01,6,510.6220,40.0,False
1408,624760835,SS-017456,78746.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MARBURY AV,CHART HOUSE ST,MIRA MESA BL,2012-05-29 06:46:03,2016-04-26 10:07:44,1971-06-01,Residential,1971-06-01,6,540.4199,40.0,False
1409,625445553,SS-010139,65576.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-02-19,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,ELDERWOOD LN,SPRING CANYON RD,ELDERWOOD CT,2013-02-19 17:11:23,2016-04-25 16:29:47,1986-06-01,Residential,1986-06-01,5,618.3855,36.0,False
1410,703454813,SS-010086,70358.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2030-01-01,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,EL HONCHO PL,AMARO DR,END,2009-11-03 13:12:15,2016-04-25 16:27:40,1973-06-01,Residential,1973-06-01,7,492.7223,36.0,False
1411,694296983,SS-015055,69656.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2016-02-29,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,KELOWNA RD,SCHNEPLE DR,DUNCANNON CT,2016-02-29 14:09:50,2016-04-26 09:24:14,1975-06-01,Residential,1975-06-01,6,264.0001,40.0,False
1412,767285743,SS-010138,65578.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-02-19,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,ELDERWOOD LN,ELDERWOOD CT,ELDERWOOD RD,2013-02-19 17:11:12,2016-05-04 08:13:03,1986-06-01,Residential,1986-06-01,5,335.9501,36.0,False
1413,366530475,SS-018039,69588.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2013-06-25,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,MENKAR RD,ALKAID DR,BELLATRIX CT,2013-06-25 11:02:27,2016-10-10 15:59:45,1973-06-01,Major,1973-06-01,6,206.0070,40.0,False
1414,125752382,SS-003607,76991.0,AC1512-JOC8,AC OVERLAY 1512-JOC8,BCWang@sandiego.gov,619-527-7500,2017-02-21,2017-07-15,2014-07-23,NaT,False,CONSTRUCTION,Overlay,AC - AC Overlay,David Williams,BAROQUE LN,DUCOS PL,PAGUERA CT,2014-07-23 10:22:14,2016-10-17 07:54:04,1978-06-01,Residential,1978-06-01,7,165.5518,36.0,False


In [ ]:
df[df['wo_id'] == 'AC1512-JOC8']

In [16]:
from datetime import datetime, timedelta
pav_csv = '/data/prod/sd_paving_datasd.csv'
dbl_spec = 2

range_start_dt = datetime.now() - timedelta(days=30)

# Read CSV
df = pd.read_csv(pav_csv)

# Multiply Length by 2x when street is over 50 feet wide
df.loc[df['WIDTH'] > 50, "LENGTH"] = (df.loc[df['WIDTH'] > 50, "LENGTH"] * 2)

# Convert to miles
df['LENGTH'] = df.LENGTH / 5280

# Convert moratorium to date
df["MORATORIUM"] = pd.to_datetime(df["MORATORIUM"])

# Get post construction, within range
mask = (df.STATUS == 'Post Construction') & \
       (df.MORATORIUM >= range_start_dt)
#df = df[mask]


df.STATUS.unique()

array(['Construction', 'Post Construction', 'BID / AWARD', 'CONSTRUCTION',
       'DESIGN', 'PLANNING'], dtype=object)

In [7]:



# Get sums
sums = df[["LENGTH", "TYPE"]].groupby("TYPE").sum()
sums.reset_index(inplace=True)

# Get total paved
total = round(sums["LENGTH"].sum(), dbl_spec)

# Get total overlay
overlay = sums.loc[sums["TYPE"] == 'Overlay', "LENGTH"].reset_index()

if len(overlay) == 0:
    overlay = 0
else:
    overlay = round(overlay["LENGTH"][0], dbl_spec)

# Get total slurry
slurry = sums.loc[sums["TYPE"] == 'Slurry', "LENGTH"].reset_index()
if len(slurry) == 0:
    slurry = 0
else:
    slurry = round(slurry["LENGTH"][0], dbl_spec)


# Return dicts
#if pav_type == 'total':
    logging.info('{} miles paved {}'.format(pav_type, total))
    #return {'value': total}
#elif pav_type == 'overlay':
    logging.info('{} miles paved {}'.format(pav_type, overlay))
    #return {'value': overlay}
#elif pav_type == 'slurry':
    logging.info('{} miles paved {}'.format(pav_type, slurry))
    #return {'value': slurry}
#else:
    #raise ValueError("Unknown pav_type")

0